# Speeding up Python with C++

## Introduction


- Python is a very nice language.
- Python has lots of libraries.
- Python is used very widely.
- Python library functions usually run very fast.

- Pure Python is very slow (interpreted language).
- To write fast code you have to wrap you ideas into the language of the library.
- What do you do if there is no library that does what you need?

- Flexibility of using Python and still being able to do exactly what you want, quickly.

## How to call C/C++ from Python?

- Python-C-API is the backbone of the standard Python interpreter, CPython. Using this API it is possible to write Python extension module in C and C++.
- CTypes is included in Python 2.5 and later. CTypes lets you talk directly to shared libraries on both Windows and UNIX.
- SWIG: Simple Wrapper Interface Generator. SWIG is capable of wrapping C in a large variety of languages.
- Cython is both a python-like language for writing C-extensions and an advanced compiler for this language.
- Pyrex is a Python-like language used to create C modules for Python.
- SIP is used to generate Python bindings for Qt (PyQt), a graphics library. It can be used to wrap any C or C++ API.
- Boost.Python lets you run C++ code from Python, and Python code from C++, seamlessly.


### Ctypes

- A foreign function library from Python. It provides C compatible data types, and allows calling functions in DLLs or shared libraries. It can be used to wrap calls to these libraries in pure Python.

#### Caveats

- Have to compile your code into a shared (dynamic) library
- Not suitable for complex data types
- No explicit support for C++ (unimportant)

#### Examples of Ctypes code:

- C types: c_int, c_double, c_float, c_bool, c_char, c_size_t...
- Array types: (c_int * 10), (c_double * 20)
- Pointer types: POINTER(c_int), POINTER(c_double)
    - Special pointer types: c_char_p, c_void_p
- Constructors: c_int(), c_int(variable), (c_int * 10)()
- Pointer to a variable: pointer(variable)
- Type casting: pointer(variable)
- Functions: sizeof(variable), sizeof(c_int), addressof(variable)

In [8]:
# Ctypes Example

# load the module
import ctypes as ct

# check the contents of the module
dir(ct)

# Ctypes data types
ct.c_double
ct.c_int
ct.c_char

ct.c_double()

# create variables using constructors
cx = ct.c_double(3.14)
cx 

# Get information about CTypes objects.
ct.sizeof(cx)
ct.addressof(cx)
hex(ct.addressof(cx))

# Ctype objects can be changed
cx.value
cx.value = 2.72
cx

c_double(2.72)

In [48]:
# Ctypes Example: Calling an External Function

# load the modules
import ctypes as ct
import ctypes.util

# check contents of ct.util
dir(ct.util)

# util.find_library() searches standard locations for libm.so
# this is a math library for c
print(ct.util.find_library("m"))

# load the library, library object with lazy access
libm = ct.cdll.LoadLibrary("libm.so.6")
dir(libm)

# find the function cos
libm.cos
dir(libm)

# Must define
# > restype is a type
# > argtype is a list of types
libm.cos.restype = ct.c_double
libm.cos.argtypes = [ct.c_double]

# calculate cos(pi)
libm.cos(3.14)

# Unfortunately you need to know the libc function. Must look through libc manual to find function

libm.so.6


-0.9999987317275395

### Problem 1)  Access double drand48(void) function from libc to get a 100 random values. https://www.gnu.org/software/libc/manual/html_mono/libc.html

In [49]:
# load the modules
import ctypes as ct
import ctypes.util

# check contents of ct.util
dir(ct.util)

# util.find_library() searches standard locations for libm.so
# this is a math library for c
print(ct.util.find_library("c"))

# load the library, library object with lazy access
libc = ct.cdll.LoadLibrary("libc.so.6")

libc.drand48
libc.drand48.restype = ct.c_double
libc.drand48.argtypes = None

rand_list = [libc.drand48(None) for i in range(100)]
print(len(rand_list))

libc.so.6
100


#### C-pointer concept refresher

- Pointer is a variable containing a generalized memory address
- Dereference operator * and Address operator &
- C-arrays and pointers are very similar
- Arrays are constants and pointers are variabls
- Pointers have types because they point to data of specific size and format
 
#### Ctypes pointers
- Arrays and pointers for accessing multiple data
- Arrays can be cast into pointers automatically
- Arrays know the limits, points do not
- There are many ways to create a pointer
- ctypes.POINTER is a constructor
- ctypes.pointer is a function


In [51]:
xx = (ctypes.c_int * 10)()
xx

In [52]:
list(xx)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [54]:
xx[:] = range(10)
xx

In [56]:
px = ctypes.POINTER(ctypes.c_int)(xx)
px

In [58]:
px[2]

2

In [62]:
# This doesnt work because its out of index
# xx[10]

In [63]:
# This doesn't exist but C doesn't care and will still spit something out from memory
px[10] 

1

#### Strings and CTypes

s = ct.c_char_p("Hello World!")
printf(s.value)

### Our Case Task

A test case python code that
- Generates a random configuration of Ar atoms inside a cubic simulation box
- Checks the configuration for spatial overlaps between atoms
- Reports the total number of overlaps in the system
- The program accepts required parameters from the command line.

### Python Code outline:

Main Code
- 3 main sections:
    - classes
    - functions
    - main code
- The main code does 5 things
    - gets input parameters
    - generates random atoms using the parameters
    - finds overlaps between atoms
    - reports the found overlaps
    - reports timings
- Timings are our data of interest

Functions and Classes
- Storage classes with some reporting capabilities
- There is a function for each major step
- Functions return a storage object for the next step
- Final timings are reported by the timing object

In [68]:
# === Classes ================================

# class timing_type:

# class params_type:

# class atom_type:

# class config_type:

# === Functions ==============================

# def get_input():
#     return params

# def gen_config(params):
#     return conf

# def check_overlaps(conf):
#     return overlaps

# === Main code ==============================
# Read input parameters.

# params = get_input()

# Generate a configuration of a Number of Ag atoms.

# conf = gen_config(params)

# Check the configuration for atomic clashes (overlaps).

# overlaps = check_overlaps(conf)

# Report info on found overlaps here.

# timings.report()

# === End of code ============================